# EDA part 2

## I. Imports & functions

In [1]:
import pandas as pd
import pycountry_convert as pc
import pycountry
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy import stats
from sklearn.preprocessing import MinMaxScaler

pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

In [2]:
data = pd.read_csv('data/final_data.csv')
data.head()

,countrycode,country,year,pop,rgdpna,delta,xr,csh_c,csh_i,csh_g,csh_x,csh_m,csh_r,rgdpna_per_cap,emp_percent,co2_emission,employment_agro,employment_industry,employment_services,export,import,inflation,net_migration,population_15_64,population_above_65,population_under_14,unemployment,urban_population,hdi
0,ALB,Albania,1990,3.286073,15105.708008,0.029161,7.562970,0.856953,0.106055,0.149501,0.042011,-0.039518,-0.115001,4596.887533,40.293633,1.840841,NaN,NaN,NaN,15.405064,24.031900,-0.431369,NaN,61.730339,5.494978,32.774683,NaN,36.428,0.650
1,ALB,Albania,1991,3.280395,10875.786133,0.028644,9.913647,0.830647,0.024768,0.251147,0.048235,-0.098372,-0.056425,3315.389193,40.161709,1.258116,57.910000,11.58,30.510000,7.484819,28.585701,35.514247,NaN,61.456989,5.648771,32.894240,16.22,36.700,0.631
2,ALB,Albania,1992,3.245886,10094.130859,0.027829,32.275396,1.149852,0.019641,0.252175,0.053268,-0.242801,-0.232135,3109.822976,32.426220,0.689859,58.110001,11.45,30.440001,12.499591,96.285881,232.984659,-443212.0,61.232496,5.827500,32.940005,16.33,37.249,0.615
3,ALB,Albania,1993,3.195199,11059.070312,0.027229,71.144097,0.993673,0.054909,0.196970,0.035555,-0.172263,-0.108844,3461.152283,31.035730,0.641406,57.619999,11.63,30.750000,15.978830,64.539503,125.650814,NaN,61.071314,6.025916,32.902771,16.52,37.799,0.618
4,ALB,Albania,1994,3.146519,11977.291016,0.026968,94.623333,0.859124,0.077424,0.208649,0.028667,-0.111053,-0.062811,3806.521116,33.970203,0.667179,57.080002,11.79,31.129999,11.983694,41.118890,35.842475,NaN,60.946907,6.230600,32.822494,16.57,38.354,0.624


## II. Imputation

In [3]:
# # resampling the data and filling missing values; getting rid of duplicates at the same time
# new_data = data.resample('60min').mean() 
# new_data = new_data.fillna(method = 'pad') # using only past data

In [4]:
# https://conferences.oreilly.com/strata/strata-ny-2019/cdn.oreillystatic.com/en/assets/1/event/300/Handling%20data%20gaps%20in%20time%20series%20using%20imputation%20Presentation.pdf
# https://www.kaggle.com/oguzkaanmavice/time-series-deal-with-missing-values
# https://impyute.readthedocs.io/en/master/api/time_series_imputation.html
# https://www.aptech.com/blog/introduction-to-the-fundamentals-of-panel-data/
# https://drive.google.com/file/d/1-NqVpF3__iKll7qvAu3Ebcx76DE-d2yH/view